In [1]:
import numpy as np
import random
import os
import cv2
from matplotlib import pyplot as plt
from numba import jit,cuda
import tensorflow as tf
from time import time
import pickle as pkl
from tensorflow.keras.utils import array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray, xyz2lab
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose
from keras.layers import Activation, Dense, Dropout, Flatten,MaxPooling2D
from keras.layers import BatchNormalization
from keras.models import Sequential
from skimage.io import imsave


In [2]:
"""
%%cmd
pip install wandb
pip install scikit-image
pip install skimage
pip install numba
pip install keras-unet

"""

'\n%%cmd\npip install wandb\npip install scikit-image\npip install skimage\npip install numba\npip install keras-unet\n\n'

In [3]:
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))


Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5



In [4]:
train_images_list =os.listdir("data/train/")
test_images_list = os.listdir("data/test/")

random.shuffle(train_images_list)
random.shuffle(test_images_list)

In [5]:
trainImageColorful = list(map(lambda x : "data/train/"+x ,train_images_list[0:6000]))
testImageColorful = list(map(lambda x : "data/test/"+x ,test_images_list[0:500]))


In [6]:
len(trainImageColorful)

6000

In [7]:
def preprocess_image_to_gray_lab(img):
    img = cv2.resize(cv2.imread(img),(400,400))
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    #img = img_to_array(load_img(img))
    img = rgb2lab(img/255)
    img = img[:,:,0]
    img = img.reshape(400, 400, 1)
    
    return img

def preprocess_image_to_lab(img):
    img = cv2.resize(cv2.imread(img),(400,400))
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    #img = img_to_array(load_img(img))
    img = rgb2lab(img/255)
    img = img[:,:,1:]
    img /= 128
    img = img.reshape(400, 400, 2)

    return img



In [8]:
trainImageColorful[0]

'data/train/3983980965.jpg'

In [9]:
start = time()

testX={}
testY={}


for ix,img in enumerate(testImageColorful):
    

   
    testX[img.split("/")[2][:-4]] = preprocess_image_to_gray_lab(img)
    testY[img.split("/")[2][:-4]] = preprocess_image_to_lab(img)
 

    if ix%100==0:
        print("Process test image - "+str(ix))

print("Time taken in seconds = ", time()-start)

Process test image - 0
Process test image - 100
Process test image - 200
Process test image - 300
Process test image - 400
Time taken in seconds =  83.92048668861389


In [10]:
with open("pickles/process_test_imagesX.pkl","wb") as process_pickle:
    pkl.dump(testX,process_pickle)
with open("pickles/process_test_imagesY.pkl","wb") as process_pickle:
    pkl.dump(testY,process_pickle)

 

In [11]:
with open("pickles/process_test_imagesX.pkl","rb") as f:
    testX = pkl.load(f)
with open("pickles/process_test_imagesY.pkl","rb") as f:
    testY = pkl.load(f)

In [12]:
start = time()

trainX={}
trainY={}


for ix, img in enumerate(trainImageColorful):
    
    img=trainImageColorful[ix]
   
    
    trainX[img.split("/")[2][:-4]] = preprocess_image_to_gray_lab(img)
    trainY[img.split("/")[2][:-4]] = preprocess_image_to_lab(img)
   
 

    if ix%100==0:
        print("Process image - "+str(ix))

print("Time taken in seconds = ", time()-start)

Process image - 0
Process image - 100
Process image - 200
Process image - 300
Process image - 400
Process image - 500
Process image - 600
Process image - 700
Process image - 800
Process image - 900
Process image - 1000
Process image - 1100
Process image - 1200
Process image - 1300
Process image - 1400
Process image - 1500
Process image - 1600
Process image - 1700
Process image - 1800
Process image - 1900
Process image - 2000
Process image - 2100
Process image - 2200
Process image - 2300
Process image - 2400
Process image - 2500
Process image - 2600
Process image - 2700
Process image - 2800
Process image - 2900
Process image - 3000
Process image - 3100
Process image - 3200
Process image - 3300
Process image - 3400
Process image - 3500
Process image - 3600
Process image - 3700
Process image - 3800
Process image - 3900
Process image - 4000
Process image - 4100
Process image - 4200
Process image - 4300
Process image - 4400
Process image - 4500


In [ ]:
with open("pickles/process_train_imagesX.pkl","wb") as process_pickle:
    pkl.dump(trainX,process_pickle)
with open("pickles/process_train_imagesY.pkl","wb") as process_pickle:
    pkl.dump(trainY,process_pickle)

MemoryError: 

In [ ]:
trainX={}
trainY={}

with open("pickles/process_train_imagesX.pkl","rb") as f:
    trainX = pkl.load(f)

with open("pickles/process_train_imagesY.pkl","rb") as f:
    trainY = pkl.load(f)

In [ ]:

def train_data_generator(trainX,trainY,trainImageColorful,num_photos_per_batch):
    
    X, y = [], []

    n = 0

    while True:

        for key in trainImageColorful:
            n +=1

            photo = trainX[key.split("/")[2][:-4]]

            photoColor = trainY[key.split("/")[2][:-4]]

          
          
                
            X.append(photo)
                   
            y.append(photoColor)
         

            if n==num_photos_per_batch:
                u = np.array(X)
                w = np.array(y)
            
             
               
                yield(u,w)
                
                X,  y = [], []
                n = 0
def valid_data_generator(validX,validY,validImageColorful,num_photos_per_batch):
    
    X, y = [], []

    n = 0

    while True:

        for key in validImageColorful:
            n +=1

            photo = validX[key.split("/")[2][:-4]]

            photoColor = validY[key.split("/")[2][:-4]]

          
          
                
            X.append(photo)
                   
            y.append(photoColor)
         

            if n==num_photos_per_batch:
                u = np.array(X)
                w = np.array(y)
            
             
               
                yield(u,w)
                
                X,  y = [], []
                n = 0
 

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
#import wandb 
#wandb.init()

In [ ]:
model = Sequential()
model.add(InputLayer(input_shape=(400, 400, 1)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', strides=2))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same' , strides=2))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same' , strides=2))
model.add(BatchNormalization())
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), activation='relu', padding='same' ))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))



In [ ]:
model.compile(optimizer='rmsprop',loss='mse' ,metrics=[
        "accuracy"
    ])
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 200, 200, 32)      320       
_________________________________________________________________
batch_normalization (BatchNo (None, 200, 200, 32)      128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 200, 200, 32)      9248      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 200, 200, 64)      18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 100, 100, 64)      36928     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 100, 100, 128)     73856     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 50, 50, 128)       1

In [ ]:
epochs = 50
number_pics_per_bath = 10
train_steps = len(trainImageColorful)//number_pics_per_bath
valid_steps = len(testImageColorful)//number_pics_per_bath

In [ ]:
from keras.models import Model,load_model
model = load_model("model_weights\model_23.h5")

In [ ]:
es = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)
filepath = "model_weights/saved-model-{epoch:02d}-{val_accuracy:.2f}.hdf5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

In [ ]:

train_generator = train_data_generator(trainX=trainX,trainY=trainY,trainImageColorful=trainImageColorful,num_photos_per_batch=number_pics_per_bath)
valid_generator = valid_data_generator(validX=testX,validY=testY,validImageColorful=testImageColorful,num_photos_per_batch=number_pics_per_bath)
 
model.fit(train_generator,epochs=epochs,steps_per_epoch=train_steps,validation_data=(valid_generator),validation_steps=valid_steps,workers=-1,verbose=1,callbacks=[es,checkpoint])
       

Epoch 1/50
  5/600 [..............................] - ETA: 7:52 - loss: 0.0118 - accuracy: 0.6419

ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted:  MemoryError: Unable to allocate 12.2 MiB for an array with shape (10, 400, 400, 1) and data type float64
Traceback (most recent call last):

  File "c:\Users\hosma\anaconda3\envs\tensorflowEnv\lib\site-packages\tensorflow\python\ops\script_ops.py", line 249, in __call__
    ret = func(*args)

  File "c:\Users\hosma\anaconda3\envs\tensorflowEnv\lib\site-packages\tensorflow\python\autograph\impl\api.py", line 645, in wrapper
    return func(*args, **kwargs)

  File "c:\Users\hosma\anaconda3\envs\tensorflowEnv\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py", line 892, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "c:\Users\hosma\anaconda3\envs\tensorflowEnv\lib\site-packages\keras\engine\data_adapter.py", line 822, in wrapped_generator
    for data in generator_fn():

  File "C:\Users\hosma\AppData\Local\Temp\ipykernel_53180\3882066652.py", line 25, in train_d

In [ ]:
def show_images(i,coloring):
    fig = plt.figure(figsize=(10, 7))
    rows = 1
    columns = 2
    fig.add_subplot(rows, columns, 1)
  
    normal = load_img(i,color_mode="grayscale",target_size=(400,400))
    plt.imshow(normal, cmap='gray', vmin=0, vmax=255)
    plt.axis('off')
    plt.title("Gray image")
    

    fig.add_subplot(rows, columns, 2)
    
  
    plt.imshow(coloring)
    plt.axis('off')
    plt.title("AI Colorizing")

def mergeOutput(x,output):
    output *= 128
  
    cur = np.zeros((400, 400, 3))
    cur[:,:,0] = x[0][:,:,0]
    cur[:,:,1:] = output[0]
    return cur

In [ ]:
for i in testImageColorful[0:5]:
    
    x = preprocess_image_to_gray_lab(i)
    
    x = x.reshape(1,400,400,1)

    output = model.predict(x)
    
    cur = mergeOutput(x,output)
    
    img = lab2rgb(cur)

   
    show_images(i,img)

   
 

In [ ]:
x = preprocess_image_to_gray_lab("test images/ataturk3.jpg")

x = x.reshape(1,400,400,1)

output = model.predict(x)
    
cur = mergeOutput(x,output)
    
img = lab2rgb(cur)

   
show_images("test images/ataturk3.jpg",img)
